## U-Net for Image Segmentation with JAX

This Colab notebook guides you through building and training a U-Net architecture using JAX for image segmentation tasks, emphasizing the use of `jax.jit` for performance optimization.

**Understanding U-Net Architecture**

U-Net is a convolutional neural network architecture specifically designed for image segmentation. Its unique U-shaped structure allows it to excel at capturing both local details (fine-grained features) and contextual information  (the broader picture) within an image.  Here's how it works:

* **Contracting Path (Encoder):**
    * Consists of multiple down-convolution blocks. Each block applies a series of convolutional layers followed by an activation function (e.g., ReLU).
    * Down-convolution layers, often with pooling operations (like max pooling), progressively reduce the spatial resolution of the input image while increasing the channels (feature maps). These extracted features become increasingly complex and high-level.

* **Expansive Path (Decoder):**
    * Composed of up-convolution blocks that utilize transposed convolutions to increase spatial resolution while decreasing channel depth.
    * Each block upsamples the feature maps, combining them with skip connections from the corresponding level in the contracting path.
    * Skip connections concatenate higher-resolution feature maps from the encoder with the expanded feature maps from the decoder. This helps recover precise localization details lost during downsampling.

* **Output Layer:**
    * A final convolutional layer, often with a sigmoid activation function, generates the segmentation output(s). This output typically  has the same dimensions as the input image.

**The benefits of this architecture make U-Net highly effective for various image segmentation tasks.**

**1. Setting Up and Importing Libraries**

In [ ]:
!pip install dm-haiku -q

In [ ]:
import jax
import jax.numpy as jnp
from jax import random

# Additional libraries you might need based on your dataset (e.g., for loading and preprocessing)
# ...
import haiku as hk
import jax.nn
from sklearn.model_selection import train_test_split
import cv2
import os
import matplotlib.pyplot as plt

**2. Defining the U-Net Architecture**

In [ ]:
@jax.jit
def conv_layer(inputs, filters, kernel_size, padding='same'):
  """
  Defines a single convolutional layer with ReLU activation and batch normalization.

  Args:
    inputs: Input tensor (e.g., image data).
    filters: Number of filters in the convolutional layer.
    kernel_size: Size of the convolutional kernel.
    padding: Padding strategy for the convolution ('same' or 'valid').

  Returns:
    Output tensor after applying convolution, ReLU activation, and batch normalization.
  """
  # Apply convolution
  conv = hk.Conv2D(filters, kernel_size, padding = padding)(inputs)

  # Apply ReLU activation (optional)
  activated = jax.nn.relu(conv)

  # Apply batch normalization (optional)
  normalized = hk.BatchNorm()(activated)

  return normalized

@jax.jit
def conv_block(inputs, filters, kernel_size):
  """Defines a convolutional block with activation and normalization (jitted)."""
  # ... implement convolutional layers, ReLU activation, and batch normalization

  conv1 = conv_layer(inputs, filters, kernel_size)
  conv2 = conv_layer(conv1, filters, kernel_size)
  output = conv2

  return output

@jax.jit
def encoder_block(inputs, filters, kernel_size):
  """Defines an encoder block with downsampling and skip connection (jitted)."""
  # ... apply two convolutional blocks
  conv = conv_block(inputs, filters, kernel_size)
  skip_connection = conv

  down_sampled = hk.MaxPool((2, 2))(conv)  # apply max pooling for downsampling

  return down_sampled, skip_connection

@jax.jit
def decoder_block(inputs, skip_connection, filters, kernel_size):
  """Defines a decoder block with upsampling and skip connection (jitted)."""
  # ... apply transposed convolution for upsampling
  t_conv = hk.Conv2DTranspose(filters, kernel_size, stride = 2, padding = 'same')(inputs)

  # ... concatenate with skip connection from encoder
  concatenated = jnp.concatenate([t_conv, skip_connection], axis=-1)

  # ... apply two convolutional blocks
  conv = conv_block(concatenated, filters, kernel_size)
  outputs = conv

  return outputs

@jax.jit
def unet(inputs, filters, kernel_size):
  """Defines the U-Net architecture (jitted)."""
  # ... create encoder blocks with increasing filter depth
  # Encoder
  enc1, skip1 = encoder_block(inputs, filters, kernel_size)
  enc2, skip2 = encoder_block(enc1, filters*2, kernel_size)
  enc3, skip3 = encoder_block(enc2, filters*4, kernel_size)

  # Bottom
  bottom = conv_block(enc3, filters*8, kernel_size)

  # ... create decoder blocks with decreasing filter depth
  # Decoder
  dec1 = decoder_block(bottom, skip3, filters*4, kernel_size)
  dec2 = decoder_block(dec1, skip2, filters*2, kernel_size)
  dec3 = decoder_block(dec2, skip1, filters, kernel_size)

  # ... apply final convolution for regression output (e.g., 4 channels for tumor segmentation)
  outputs = conv_layer(dec3, 4, 1)

  return outputs

**3. Data Acquisition and Preprocessing:**

* Download the BraTS 2020 dataset from [https://www.kaggle.com/datasets/mateuszbuda/lgg-mri-segmentation](https://www.kaggle.com/datasets/mateuszbuda/lgg-mri-segmentation).
* Preprocess the data by:
    * Resizing images to a fixed size suitable for your model.
    * Normalizing pixel intensities (e.g., scaling between 0 and 1).
    * Segmenting the brain region using provided masks if necessary.
    * Splitting the data into training, validation, and test sets.


In [ ]:
def load_data(image_folder):
    image_paths = sorted([os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith('.tif')])

    images = [cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) for img_path in image_paths]

    return jnp.array(images)

In [ ]:
def preprocess_images(images):
    # Resize images to a fixed size suitable for your model
    resized_images = [cv2.resize(img, (256, 256)) for img in images]
    # Convert images to grayscale
    grayscale_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in resized_images]
    # Normalize pixel intensities (scaling between 0 and 1)
    normalized_images = [img / 255.0 for img in grayscale_images]
    return jnp.array(normalized_images)

In [ ]:
# Load your image segmentation dataset (modify accordingly)
# ...
image_folder = "datasets/images"
mask_folder = "datasets/masks"

# Load data
images = load_data(image_folder)
masks = load_data(mask_folder)

# Preprocess data
processed_images = preprocess_images(images)
processed_masks = preprocess_images(masks)

# Preprocess data (normalize and resize). Then split into train, test and validation:

# Split into train and validation:
train_images, test_images, train_masks, test_masks = train_test_split(processed_images,
                                                                    processed_masks,
                                                                    test_size=0.2,
                                                                    random_state=42)

# If you want to further split the training set into training and testing sets, you can do so like this:
train_images, val_images, train_masks, val_masks = train_test_split(train_images,
                                                                      train_masks,
                                                                      test_size=0.2,
                                                                      random_state=42)

**4. Model Training**

In [ ]:
# Hyperparameters
learning_rate = 0.001
epochs = 50

# Initialize model parameters
key = random.PRNGKey(0)

input_shape = (1, 256, 256, 1)
unet_model = hk.transform(unet)
params = unet_model.init(key, jnp.zeros(input_shape, jnp.float32))

In [ ]:
# Loss function (e.g., mean squared error for each segmentation channel)
def loss_fn(params, images, masks):
    logits = unet_model.apply(params, None, images)
    return jnp.mean((logits - masks)**2)

In [ ]:
@jax.jit
def train_step(params, images, masks):
  """Training step with jitted loss and gradient calculation (jitted)."""
  # ... calculate loss and gradients
  # ... update model parameters using SGD optimizer

  grad_fn = jax.grad(loss_fn)
  grads = grad_fn(params, images, masks)
  updated_params = jax.tree_multimap(lambda p, g: p - learning_rate * g, params, grads)

  return updated_params

In [ ]:
# Assuming train_images and train_masks are defined and preprocessed
for epoch in range(epochs):
  # ... training loop using train_step function
  # ... (consider logging training progress or visualizing intermediate results)

  for i in range(0, len(train_images), 32):
          batch_images = train_images[i:i+32]
          batch_masks = train_masks[i:i+32]

          params = train_step(params, batch_images, batch_masks)

          if i % 100 == 0:
              loss_value = loss_fn(params, batch_images, batch_masks)
              print(f"Epoch {epoch}, Step {i}, Loss: {loss_value}")

print("Training completed!")

**5. Model Evaluation**

In [ ]:
@jax.jit
def dice_coefficient(pred, target):
    """Compute Dice coefficient."""
    intersection = jnp.sum(pred * target)
    union = jnp.sum(pred) + jnp.sum(target)
    return (2. * intersection) / (union + 1e-7)

In [ ]:
@jax.jit
def jaccard_index(pred, target):
    """Compute Jaccard index."""
    intersection = jnp.sum(pred * target)
    union = jnp.sum(pred) + jnp.sum(target) - intersection
    return (intersection) / (union + 1e-7)

In [ ]:
@jax.jit
def evaluate(params, images, masks):
  """Evaluation with jitted metric calculation (jitted)."""
  logits = unet_model.apply(params, None, images)

  # Convert logits to binary predictions
  predictions = jnp.argmax(logits, axis=-1)
  predictions = jax.nn.one_hot(predictions, 4)

  # Compute metrics for each segmentation channel
  dice_scores = jnp.array([dice_coefficient(predictions[:, :, :, i], masks[:, :, :, i]) for i in range(4)])
  jaccard_scores = jnp.array([jaccard_index(predictions[:, :, :, i], masks[:, :, :, i]) for i in range(4)])

  return {"Dice": dice_scores, "Jaccard": jaccard_scores}

In [ ]:
val_metrics = evaluate(params, val_images, val_masks)
print(f"Validation metrics: {val_metrics}")

In [ ]:
# Visualization function
def visualize_results(images, masks, predictions, num_samples=5):
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 15))

    for i in range(num_samples):
        axes[i, 0].imshow(images[i].squeeze(), cmap='gray')
        axes[i, 0].set_title("Input Image")
        axes[i, 0].axis("off")

        axes[i, 1].imshow(masks[i].squeeze(), cmap='gray')
        axes[i, 1].set_title("True Mask")
        axes[i, 1].axis("off")

        axes[i, 2].imshow(predictions[i].squeeze(), cmap='gray')
        axes[i, 2].set_title("Predicted Mask")
        axes[i, 2].axis("off")

    plt.show()

In [ ]:
# Assuming params and test_images are defined
test_logits = unet_model.apply(params, None, test_images)
test_predictions = jnp.argmax(test_logits, axis=-1)
visualize_results(test_images[:5], test_masks[:5], test_predictions[:5])

**6. Summary and Next Steps**

This notebook provides a foundation for building and training a U-Net model with JAX for image segmentation while emphasizing the importance of `jax.jit` for performance optimization. Remember to:

* Replace the placeholders in the code with appropriate JAX operations and functions based on your chosen dataset and architecture details.
* Experiment with different hyperparameters (learning rate, filters, etc.) and training strategies to improve the model's performance.
* Explore advanced techniques like data augmentation and regularization to enhance modelgeneralizability and robustness.

By completing this notebook and understanding the U-Net architecture, you can gain valuable practical experience in building and training deep learning models for image segmentation tasks using JAX effectively.